# Complete Numbers ($\mathbb{U}$) Implementation

This notebook demonstrates the practical implementation of Complete Numbers, a new mathematical structure introduced in the paper "The Set of Vanished Numbers $\mathbb{U}$" (Spencer, 2024). The structure is designed to track information that is lost during multiplication by zero while maintaining computational tractability. Beginning with a foundational singleton set containing a pure zero element, we develop a rigorous framework of mappings that preserve this otherwise lost information.

> **Note**: This is a companion implementation to a paper under consideration. The complete theoretical treatment, including proofs and mathematical construction, can be found in the [GitHub repository](https://github.com/YOUR_USERNAME/complete-numbers/paper/vanished_numbers.pdf).

## Mathematical Background

The multiplicative property of zero stands as one of the most fundamental yet peculiar features of computational systems. When a number is multiplied by zero, the result is unambiguous - the product is zero. However, this apparently simple operation masks a subtle but significant loss of information. In the expression $a \times 0 = 0$, all information about the original number $a$ vanishes in the result.

Consider the sequence:
\begin{align*}
    5 \times 0 &= 0 \\
    3 \times 0 &= 0 \\
    \therefore 5 \times 0 &= 3 \times 0
\end{align*}

While logically consistent, this sequence demonstrates how distinct numerical values become indistinguishable after multiplication by zero. This property can create significant challenges in computational systems and data structure design. The complete number system provides a framework that preserves this information while maintaining consistency with standard arithmetic through explicit mappings.

## Implementation

The implementation patterns of Complete Numbers parallel those of complex numbers in Python. Consider how complex numbers handle zero multiplication:

```python
# Complex Number Patterns
z = 3 + 4j               # Create complex number
z = complex(3, 4)        # Alternative construction
z.real * 0  -> 0.0       # Real component times zero
z.imag * 0  -> 0.0       # Imaginary component times zero
z * 0       -> 0j        # Whole complex number times zero
```

Complete Numbers follow similar patterns but preserve the vanished structure:

```python
# Complete Number Patterns
cu = 3 + 4j               # Create complete number
cu = CompleteNumber(3, 4) # Alternative construction
cu.real * 0  -> 3u        # Real component times zero
cu.imag * 0  -> 4uj       # Imaginary component times zero
cu * 0      -> 3u + 4uj   # Whole complete number times zero
```

Now we implement the core Complete Numbers classes that realize this behavior:

In [2]:
from typing import Union, Tuple
import numbers
import cmath

class CompleteComponent:
    """Drop-in replacement for float that maintains absorption into U"""
    def __init__(self, value, component_type):
        self._value = float(value)
        self._type = component_type    # 'real' or 'imag'

    def __mul__(self, other):
        if other == 0:
            if self._type == 'real':
                # Real component times zero -> just the absorbed value
                return f"{self._value}u"
            else:  # imaginary
                # Imaginary component times zero -> just the absorbed value
                return f"{self._value}uj"
        else:
            # Non-zero multiplication - return just the float value
            return self._value * other

    def __truediv__(self, other):
        """Component division behaves like regular float division"""
        if other == 0:
            raise ZeroDivisionError("float division by zero")
        return self._value / other

    def __rtruediv__(self, other):
        if self._value == 0:
            raise ZeroDivisionError("float division by zero")
        return other / self._value

    def __rmul__(self, other):
        return self.__mul__(other)

    def __repr__(self):
        return str(self._value)

    def __float__(self):
        return self._value

class CompleteNumber:
    def __init__(self, real, imag, u_real=0, u_imag=0):
        self._real = float(real)
        self._imag = float(imag)
        self._u_real = float(u_real)
        self._u_imag = float(u_imag)

    @property
    def real(self):
        return CompleteComponent(self._real, 'real')

    @property
    def imag(self):
        return CompleteComponent(self._imag, 'imag')

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            if other == 0:
                # Whole number times zero -> both components go to U
                return CompleteNumber(0, 0, self._real, self._imag)
            else:
                # Regular multiplication
                return CompleteNumber(self._real * other, self._imag * other)
        return NotImplemented

    def __truediv__(self, other):
        """Complete number division"""
        if isinstance(other, (int, float)):
            if other == 0:
                # If there are any non-absorbed components, division by zero is undefined
                if self._real != 0 or self._imag != 0:
                    raise ZeroDivisionError("division by zero")
                # Only absorbed components present
                return CompleteNumber(
                    self._u_real,
                    self._u_imag
                )
            else:
                # Regular division
                return CompleteNumber(
                    self._real / other,
                    self._imag / other,
                    self._u_real / other,
                    self._u_imag / other
                )
        return NotImplemented

    def __rmul__(self, other):
        return self.__mul__(other)

    def __repr__(self):
        parts = []
        if self._real != 0 or (self._imag == 0 and self._u_real == 0 and self._u_imag == 0):
            parts.append(str(self._real))
        if self._imag != 0:
            parts.append(f"{self._imag}j")
        if self._u_real != 0:
            parts.append(f"{self._u_real}u")
        if self._u_imag != 0:
            parts.append(f"{self._u_imag}uj")

        return " + ".join(parts).replace(" + -", " - ")

## Basic Operations

Let's explore how Complete Numbers behave under basic operations:

In [3]:
# Create a complete number
print("Creating a complete number (3 + 4j):")
cn = CompleteNumber(3, 4)
print(f"cn = {cn}\n")

# Demonstrate multiplication by zero
print("Multiplication by zero (preserves structure):")
cn_zero = cn * 0
print(f"cn * 0 = {cn_zero}\n")

# Recover original values
print("Recovering original values:")
cn_recovered = cn_zero / 0
print(f"cn_zero / 0 = {cn_recovered}")

Creating a complete number (3 + 4j):
cn = 3.0 + 4.0j

Multiplication by zero (preserves structure):
cn * 0 = 3.0u + 4.0uj

Recovering original values:
cn_zero / 0 = 3.0 + 4.0j


## Vanishing Summation Structure

Now let's demonstrate how Complete Numbers preserve information through the vanishing of the summation structure:

In [4]:
def demonstrate_vanishing_summation():
    """Demonstrates how multiplication by zero represents the vanishing of the summation
    structure itself, not the removal of quantities."""

    # Create complete numbers for our quantities
    box_quantity = CompleteNumber(5, 0)    # 5 as a quantity
    crate_quantity = CompleteNumber(3, 0)  # 3 as a quantity
    print(f"Initial quantities:")
    print(f"Box:   {box_quantity}")
    print(f"Crate: {crate_quantity}\n")

    # Demonstrate multiplication by zero (vanishing of summation structure)
    box_vanished = box_quantity * 0
    crate_vanished = crate_quantity * 0
    print(f"After multiplication by zero (vanishing of summation structure):")
    print(f"Box:   {box_vanished}")    # Should be 5u - preserving the vanished structure
    print(f"Crate: {crate_vanished}\n") # Should be 3u - preserving the vanished structure

    # Add the vanished quantities (combining our structural information)
    total_vanished = CompleteNumber(
        0, 0,
        box_vanished._u_real + crate_vanished._u_real,
        box_vanished._u_imag + crate_vanished._u_imag
    )
    print(f"Combined vanished structures:")
    print(f"Total: {total_vanished}\n")

    # Recover the original total
    total = total_vanished / 0
    print(f"Recovered quantity from vanished structure:")
    print(f"Total: {total}")

    return box_vanished, crate_vanished, total_vanished

demonstrate_vanishing_summation()

Initial quantities:
Box:   5.0
Crate: 3.0

After multiplication by zero (vanishing of summation structure):
Box:   5.0u
Crate: 3.0u

Combined vanished structures:
Total: 8.0u

Recovered quantity from vanished structure:
Total: 8.0


(5.0u, 3.0u, 8.0u)

## Component-wise Operations

Complete Numbers allow us to track the vanished real and imaginary components separately:

In [5]:
# Create a complete number
cn = CompleteNumber(3, 4)
print(f"Complete number: {cn}\n")

# Real component operations
print("Real component:")
print(f"cn.real = {cn.real}")
print(f"cn.real * 0 = {cn.real * 0}\n")

# Imaginary component operations
print("Imaginary component:")
print(f"cn.imag = {cn.imag}")
print(f"cn.imag * 0 = {cn.imag * 0}")

Complete number: 3.0 + 4.0j

Real component:
cn.real = 3.0
cn.real * 0 = 3.0u

Imaginary component:
cn.imag = 4.0
cn.imag * 0 = 4.0uj


## Explore Further

Try experimenting with your own Complete Number operations. Here are some suggestions:

1. Create Complete Numbers with different real and imaginary components
2. Perform operations and observe how structure is preserved
3. Try combining multiple vanished structures
4. Experiment with recovery operations